In [1]:
import pandas as pd
from google.cloud import storage
import os

In [2]:
# Establecer la ruta del archivo de credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../cloud/proyecto2-462217-118002bf0caf.json"

In [3]:
bucket_name = "mi-bucket-heartdata"
blob_path = "data/heart-disease-dataset.csv"
file_path = f"gs://{bucket_name}/{blob_path}"

In [4]:
df = pd.read_csv(file_path)

In [5]:
df.head()

,age,sex,chest_pain_type,resting_bp_s,cholesterol,fasting_blood_sugar,resting_ecg,max_heart_rate,exercise_angina,oldpeak,st_slope,target
0,40.0,1.0,2.0,140.0,289.0,0.0,0.0,172.0,0.0,0.0,1.0,0.0
1,49.0,0.0,3.0,160.0,180.0,0.0,0.0,156.0,0.0,1.0,2.0,1.0
2,37.0,1.0,2.0,130.0,283.0,0.0,1.0,98.0,0.0,0.0,1.0,0.0
3,48.0,0.0,4.0,138.0,214.0,0.0,0.0,108.0,1.0,1.5,2.0,1.0
4,54.0,1.0,3.0,150.0,195.0,0.0,0.0,122.0,0.0,0.0,1.0,0.0


Eliminamos datos duplicados

In [6]:
df = df.drop_duplicates()

Eliminamos outliers

In [7]:
# Filtrado de outliers basado en rangos definidos
df_clean = df[
    (df['age'] >= 18) & (df['age'] <= 100) &
    (df['resting_bp_s'] >= 90) & (df['resting_bp_s'] <= 210) &
    (df['cholesterol'] >= 100) & (df['cholesterol'] <= 700) &
    (df['max_heart_rate'] >= 60) & (df['max_heart_rate'] <= 220) &
    (df['oldpeak'] >= 0) & (df['oldpeak'] <= 6)
].copy()

In [8]:
df_clean.shape

(743, 12)

In [10]:
# Guardar el DataFrame limpio en un nuevo archivo CSV
df_clean.to_csv("../../data/clean/clean_heart-disease-dataset.csv", index=False)

In [11]:
def upload_csv_to_bucket(bucket_name, source_file_path, destination_blob_name):
    """
    Sube un archivo CSV a un bucket de GCP.

    bucket_name: nombre del bucket (str)
    source_file_path: ruta local del archivo CSV (str)
    destination_blob_name: nombre con el que se guardará en el bucket (str)
    """
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_path)

    print(f"Archivo {source_file_path} subido a gs://{bucket_name}/{destination_blob_name}")

In [12]:
upload_csv_to_bucket(
    bucket_name="mi-bucket-heartdata",
    source_file_path="../../data/clean/clean_heart-disease-dataset.csv",
    destination_blob_name="data/clean_heart-disease-dataset.csv"
)

Archivo ../../data/clean/clean_heart-disease-dataset.csv subido a gs://mi-bucket-heartdata/data/clean_heart-disease-dataset.csv
